In [1]:
import selenium
from bs4 import BeautifulSoup
import urllib2
import json

import re
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time

import pandas as pd
import numpy as np

## Define values for combinations

In [2]:
CAR_MODEL = ['TOYOTA COROLLA ALTIS','HONDA CIVIC']
CAR_ENGINE = ['1600','1700']
CAR_REGIS = ['2016','2017']
OFF_PEAK = ['YES','NO'] 
GENDER = ['MALE','FEMALE']
MARITAL = ['SINGLE','MARRIED']
BIRTHDATE = ['01/01/1993','01/01/1983']
DRI_EXP = ['LESS_TWO','MORE_TWO']
CLM_PST = ['YES','NO']
#NCD = [0,20]

## Create combination dataframe

In [5]:
df_model = pd.DataFrame(data = ['TOYOTA COROLLA ALTIS','HONDA CIVIC'], columns=['CAR_MODEL'])
df_model['tmp_key'] = 1

df_engine = pd.DataFrame(data = ['1600','1700'], columns=['CAR_ENGINE'])
df_engine['tmp_key'] = 1

df_regis = pd.DataFrame(data = ['2016','2017'], columns=['CAR_REGIS'])
df_regis['tmp_key'] = 1

df_offpeak = pd.DataFrame(data = ['YES','NO'], columns=['OFF_PEAK'])
df_offpeak['tmp_key'] = 1

df_gender = pd.DataFrame(data = ['MALE','FEMALE'], columns=['GENDER'])
df_gender['tmp_key'] = 1

df_marital = pd.DataFrame(data = ['SINGLE','MARRIED'], columns=['MARITAL'])
df_marital['tmp_key'] = 1

df_birthdate = pd.DataFrame(data = ['01/01/1993','01/01/1983'], columns=['BIRTHDATE'])
df_birthdate['tmp_key'] = 1

df_driexp = pd.DataFrame(data = ['LESS_TWO','MORE_TWO'], columns=['DRI_EXP'])
df_driexp['tmp_key'] = 1

df_clmpst = pd.DataFrame(data = ['NO'], columns=['CLM_PST'])
df_clmpst['tmp_key'] = 1

In [6]:
df = df_model.merge(df_engine,how='outer')

In [7]:
for tmp_df in [df_regis, df_offpeak, df_gender, df_marital, df_birthdate, df_driexp, df_clmpst]:
    df = df.merge(tmp_df, how='outer')

In [14]:
df['plan_name'] = 'Drivo Classic Plan'
df['plan_quotes'] = 0.0

## Scrap for 10 combinations for test

In [8]:
df_test = df.sample(10).reset_index(drop=True)

In [3]:
#initialize selenium chrom driver
path_to_chromedriver = 'C:\Users\liuleo\Documents\Python\chromedriver_win32\chromedriver.exe' 
browser = webdriver.Chrome(executable_path = path_to_chromedriver)

# navigate to the webpage containing information to be crawled
url = 'https://insurance.income.com.sg/quote/car/index/'
browser.get(url)

In [12]:
for i in range(len(df_test)):
    print df_test.loc[i, 'CAR_MODEL'], df.loc[i, 'CAR_ENGINE'], df.loc[i, 'DRI_EXP'], df.loc[i, 'CLM_PST']
    # select car model 
    select_car_model = Select(browser.find_element_by_id('carMakeModel'))
    select_car_model.select_by_visible_text(df_test.loc[i, 'CAR_MODEL'])
    
    # select car engine
    select_car_eng = Select(browser.find_element_by_id('engineCapacities_car'))
    select_car_eng.select_by_visible_text(df_test.loc[i, 'CAR_ENGINE'])
    
    # sleep before input text 
    time.sleep(1)
    #input car registered year
    browser.find_element_by_xpath('//*[@id="input-moto-year"]').send_keys(df_test.loc[i, 'CAR_REGIS'])
    
    # select Off-Peak Car Scheme (default value is no)
    if df_test.loc[i, 'OFF_PEAK'] == 'YES':
        browser.find_element_by_xpath('//*[@id="tab1"]/div/div[4]/div/div/label[1]').click()
    #else:
    #    browser.find_element_by_xpath('//*[@id="tab1"]/div/div[4]/div/div/label[2]').click()
    
    #click next
    browser.find_element_by_xpath('//*[@id="tab1"]/div/ul/li/a').click()
    # sleep when goes to a new page
    time.sleep(1)
    
    #select gender (default is male)
    if df_test.loc[i, 'GENDER'] == 'FEMALE':
        browser.find_element_by_xpath('//*[@id="tab2"]/div/div[1]/div/div[1]/label[2]').click()        
    #else:
    #    browser.find_element_by_xpath('//*[@id="tab2"]/div/div[1]/div/div[1]/label[1]').click()
    
    #select marital status (default single)
    if df_test.loc[i, 'MARITAL'] == 'MARRIED':
        browser.find_element_by_xpath('//*[@id="tab2"]/div/div[2]/div/div[1]/label[2]').click()
    #else:
    #    browser.find_element_by_xpath('//*[@id="tab2"]/div/div[2]/div/div[1]/label[1]').click()
    
    time.sleep(2)
    
    #input date of birth
    browser.find_element_by_xpath('//*[@id="date"]').send_keys(df_test.loc[i, 'BIRTHDATE'])
    
    #click next
    browser.find_element_by_xpath('//*[@id="tab2"]/div/ul/li[2]/a').click()
    # sleep when goes to a new page
    time.sleep(1)
    
    #choose driver years exp (no default value)
    if df_test.loc[i, 'DRI_EXP'] == 'LESS_TWO':
        browser.find_element_by_xpath('//*[@id="tab3"]/div/div[1]/div/div[1]/label[1]').click()
    else:
        browser.find_element_by_xpath('//*[@id="tab3"]/div/div[1]/div/div[1]/label[2]').click()
    
    #choos flag for claim for last 3 years (if yes, then provide name and phone, qutoes will be given after call)
    #if df_test.loc[i, 'CLM_PST'] == 'YES':
    #    browser.find_element_by_xpath('//*[@id="tab3"]/div/div[2]/div/div/label[2]').click()
        
    #click next
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="tab3"]/div/ul/li[2]/a').click()
    time.sleep(1)
    
    #click confirm and get the quote
    browser.find_element_by_xpath('//*[@id="tab4"]/div[1]/button').click()
    
    time.sleep(4)
    html = browser.page_source
    # pass to BeautifulSoup parser
    soup = BeautifulSoup(html, "lxml")
    
    # Get basic plan
    name_box = soup.find_all('span', attrs={'class': 'plan-title'})

    basic_plan_name = name_box[0].text.strip()
    print basic_plan_name
    
    df_test.loc[i,'plan_name'] = basic_plan_name
    
    # Get price
    price_box = soup.find_all('p',attrs={'id':'plan_price_before_GST_1'})
    price = price_box[0].text.strip()
    
    df_test.loc[i,'plan_quotes'] = price
    
    time.sleep(1)
    # start a new quote
    url = 'https://insurance.income.com.sg/quote/car/index/'
    browser.get(url)
    # accept the alert 
    #browser.switch_to_alert().accept()
    time.sleep(1)

HONDA CIVIC 1600 LESS_TWO NO
Drivo Classic Plan
TOYOTA COROLLA ALTIS 1600 MORE_TWO NO
Drivo Classic Plan
HONDA CIVIC 1600 LESS_TWO NO
Drivo Classic Plan
TOYOTA COROLLA ALTIS 1600 MORE_TWO NO
Drivo Classic Plan
HONDA CIVIC 1600 LESS_TWO NO
Drivo Classic Plan
TOYOTA COROLLA ALTIS 1600 MORE_TWO NO
Drivo Classic Plan
TOYOTA COROLLA ALTIS 1600 LESS_TWO NO
Drivo Classic Plan
HONDA CIVIC 1600 MORE_TWO NO
Drivo Classic Plan
HONDA CIVIC 1600 LESS_TWO NO
Drivo Classic Plan
HONDA CIVIC 1600 MORE_TWO NO
Drivo Classic Plan


## Scrap all combinations (2^8 ~ 256)

In [ ]:
#initialize selenium chrom driver
path_to_chromedriver = 'C:\Users\liuleo\Documents\Python\chromedriver_win32\chromedriver.exe' 
browser = webdriver.Chrome(executable_path = path_to_chromedriver)

# navigate to the webpage containing information to be crawled
url = 'https://insurance.income.com.sg/quote/car/index/'
browser.get(url)

In [16]:
for i in range(len(df)):
    print df.loc[i, 'CAR_MODEL'], df.loc[i, 'CAR_ENGINE'], df.loc[i, 'DRI_EXP'], df.loc[i, 'CLM_PST']
    # select car model 
    select_car_model = Select(browser.find_element_by_id('carMakeModel'))
    select_car_model.select_by_visible_text(df.loc[i, 'CAR_MODEL'])
    
    # select car engine
    select_car_eng = Select(browser.find_element_by_id('engineCapacities_car'))
    select_car_eng.select_by_visible_text(df.loc[i, 'CAR_ENGINE'])
    
    # sleep before input text 
    time.sleep(1)
    #input car registered year
    browser.find_element_by_xpath('//*[@id="input-moto-year"]').send_keys(df.loc[i, 'CAR_REGIS'])
    
    # select Off-Peak Car Scheme (default value is no)
    if df.loc[i, 'OFF_PEAK'] == 'YES':
        browser.find_element_by_xpath('//*[@id="tab1"]/div/div[4]/div/div/label[1]').click()
    #else:
    #    browser.find_element_by_xpath('//*[@id="tab1"]/div/div[4]/div/div/label[2]').click()
    
    #click next
    browser.find_element_by_xpath('//*[@id="tab1"]/div/ul/li/a').click()
    # sleep when goes to a new page
    time.sleep(1)
    
    #select gender (default is male)
    if df.loc[i, 'GENDER'] == 'FEMALE':
        browser.find_element_by_xpath('//*[@id="tab2"]/div/div[1]/div/div[1]/label[2]').click()        
    #else:
    #    browser.find_element_by_xpath('//*[@id="tab2"]/div/div[1]/div/div[1]/label[1]').click()
    
    #select marital status (default single)
    if df.loc[i, 'MARITAL'] == 'MARRIED':
        browser.find_element_by_xpath('//*[@id="tab2"]/div/div[2]/div/div[1]/label[2]').click()
    #else:
    #    browser.find_element_by_xpath('//*[@id="tab2"]/div/div[2]/div/div[1]/label[1]').click()
    
    time.sleep(2)
    
    #input date of birth
    browser.find_element_by_xpath('//*[@id="date"]').send_keys(df.loc[i, 'BIRTHDATE'])
    
    #click next
    browser.find_element_by_xpath('//*[@id="tab2"]/div/ul/li[2]/a').click()
    # sleep when goes to a new page
    time.sleep(1)
    
    #choose driver years exp (no default value)
    if df.loc[i, 'DRI_EXP'] == 'LESS_TWO':
        browser.find_element_by_xpath('//*[@id="tab3"]/div/div[1]/div/div[1]/label[1]').click()
    else:
        browser.find_element_by_xpath('//*[@id="tab3"]/div/div[1]/div/div[1]/label[2]').click()
    
    #choos flag for claim for last 3 years (if yes, then provide name and phone, qutoes will be given after call)
    #if df.loc[i, 'CLM_PST'] == 'YES':
    #    browser.find_element_by_xpath('//*[@id="tab3"]/div/div[2]/div/div/label[2]').click()
        
    #click next
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="tab3"]/div/ul/li[2]/a').click()
    time.sleep(1)
    
    #click confirm and get the quote
    browser.find_element_by_xpath('//*[@id="tab4"]/div[1]/button').click()
    
    time.sleep(4)
    html = browser.page_source
    # pass to BeautifulSoup parser
    soup = BeautifulSoup(html, "lxml")
    
    # Get basic plan
    name_box = soup.find_all('span', attrs={'class': 'plan-title'})

    basic_plan_name = name_box[0].text.strip()
    print basic_plan_name
    
    df.loc[i,'plan_name'] = basic_plan_name
    
    # Get price
    price_box = soup.find_all('p',attrs={'id':'plan_price_before_GST_1'})
    price = price_box[0].text.strip()
    
    df.loc[i,'plan_quotes'] = price
    
    time.sleep(1)
    # start a new quote
    url = 'https://insurance.income.com.sg/quote/car/index/'
    browser.get(url)
    # accept the alert 
    #browser.switch_to_alert().accept()
    time.sleep(1)

TOYOTA COROLLA ALTIS 1600 LESS_TWO NO
Drivo Classic Plan
TOYOTA COROLLA ALTIS 1600 MORE_TWO NO
Drivo Classic Plan
TOYOTA COROLLA ALTIS 1600 LESS_TWO NO
Drivo Classic Plan
TOYOTA COROLLA ALTIS 1600 MORE_TWO NO
Drivo Classic Plan
TOYOTA COROLLA ALTIS 1600 LESS_TWO NO
Drivo Classic Plan
TOYOTA COROLLA ALTIS 1600 MORE_TWO NO
Drivo Classic Plan
TOYOTA COROLLA ALTIS 1600 LESS_TWO NO
Drivo Classic Plan
TOYOTA COROLLA ALTIS 1600 MORE_TWO NO
Drivo Classic Plan
TOYOTA COROLLA ALTIS 1600 LESS_TWO NO
Drivo Classic Plan
TOYOTA COROLLA ALTIS 1600 MORE_TWO NO
Drivo Classic Plan
TOYOTA COROLLA ALTIS 1600 LESS_TWO NO
Drivo Classic Plan
TOYOTA COROLLA ALTIS 1600 MORE_TWO NO
Drivo Classic Plan
TOYOTA COROLLA ALTIS 1600 LESS_TWO NO
Drivo Classic Plan
TOYOTA COROLLA ALTIS 1600 MORE_TWO NO
Drivo Classic Plan
TOYOTA COROLLA ALTIS 1600 LESS_TWO NO
Drivo Classic Plan
TOYOTA COROLLA ALTIS 1600 MORE_TWO NO
Drivo Classic Plan
TOYOTA COROLLA ALTIS 1600 LESS_TWO NO
Drivo Classic Plan
TOYOTA COROLLA ALTIS 1600 MORE_

HONDA CIVIC 1600 MORE_TWO NO
Drivo Classic Plan
HONDA CIVIC 1600 LESS_TWO NO
Drivo Classic Plan
HONDA CIVIC 1600 MORE_TWO NO
Drivo Classic Plan
HONDA CIVIC 1600 LESS_TWO NO
Drivo Classic Plan
HONDA CIVIC 1600 MORE_TWO NO
Drivo Classic Plan
HONDA CIVIC 1600 LESS_TWO NO
Drivo Classic Plan
HONDA CIVIC 1600 MORE_TWO NO
Drivo Classic Plan
HONDA CIVIC 1600 LESS_TWO NO
Drivo Classic Plan
HONDA CIVIC 1600 MORE_TWO NO
Drivo Classic Plan
HONDA CIVIC 1600 LESS_TWO NO
Drivo Classic Plan
HONDA CIVIC 1600 MORE_TWO NO
Drivo Classic Plan
HONDA CIVIC 1600 LESS_TWO NO
Drivo Classic Plan
HONDA CIVIC 1600 MORE_TWO NO
Drivo Classic Plan
HONDA CIVIC 1600 LESS_TWO NO
Drivo Classic Plan
HONDA CIVIC 1600 MORE_TWO NO
Drivo Classic Plan
HONDA CIVIC 1600 LESS_TWO NO
Drivo Classic Plan
HONDA CIVIC 1600 MORE_TWO NO
Drivo Classic Plan
HONDA CIVIC 1600 LESS_TWO NO
Drivo Classic Plan
HONDA CIVIC 1600 MORE_TWO NO
Drivo Classic Plan
HONDA CIVIC 1600 LESS_TWO NO
Drivo Classic Plan
HONDA CIVIC 1600 MORE_TWO NO
Drivo Class

In [25]:
df.to_csv('C:/Users/liuleo/Documents/KT/WEB_SCRAP/ntuc_ex.csv',index=False, sep='|')

In [ ]:
# select car model 
select_car_model = Select(browser.find_element_by_id('carMakeModel'))
select_car_model.select_by_visible_text('TOYOTA COROLLA ALTIS')

In [ ]:
# select car engine
select_car_eng = Select(browser.find_element_by_id('engineCapacities_car'))
select_car_eng.select_by_visible_text('1600')

In [ ]:
# sleep before input text 
time.sleep(1)
#input car registered year
browser.find_element_by_xpath('//*[@id="input-moto-year"]').send_keys('2016')

In [ ]:
# select Off-Peak Car Scheme
browser.find_element_by_xpath('//*[@id="tab1"]/div/div[4]/div/div/label[1]').click()

In [ ]:
#click next
browser.find_element_by_xpath('//*[@id="tab1"]/div/ul/li/a').click()
# sleep when goes to a new page
time.sleep(1)

In [ ]:
# male and single. if change, use corresponding xpath
time.sleep(1)
#input date of birth
browser.find_element_by_xpath('//*[@id="date"]').send_keys('01/01/1993')

In [ ]:
#click next
browser.find_element_by_xpath('//*[@id="tab2"]/div/ul/li[2]/a').click()
time.sleep(1)

In [ ]:
#choose more than 2 years exp
browser.find_element_by_xpath('//*[@id="tab3"]/div/div[1]/div/div[1]/label[2]').click()

#for claims in past 3 years, choose no (default value)

#for NCD value (keep 0% for default)
# if need i times 10% values, then run the following with i times
#browser.find_element_by_xpath('//*[@id="tab3"]/div/div[3]/div/div[1]/span[2]/button').click()

#date
# start from the following day and contract lenght one year

In [ ]:
#click next
browser.find_element_by_xpath('//*[@id="tab3"]/div/ul/li[2]/a').click()
time.sleep(1)

In [ ]:
#click confirm and get the quote
browser.find_element_by_xpath('//*[@id="tab4"]/div[1]/button').click()


In [ ]:
time.sleep(1)
html = browser.page_source
# pass to BeautifulSoup parser
soup = BeautifulSoup(html, "lxml")

In [ ]:
# Get basic plan
name_box = soup.find_all('span', attrs={'class': 'plan-title'})

basic_plan_name = name_box[0].text.strip()
print basic_plan_name

In [ ]:
# Get price
price_box = soup.find_all('p',attrs={'id':'plan_price_before_GST_1'})
print price_box[0].text.strip()

In [ ]:
time.sleep(1)
# start a new quote
url = 'https://insurance.income.com.sg/quote/car/index/'
browser.get(url)
# accept the alert 
#browser.switch_to_alert().accept()
time.sleep(1)